# Modeling

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.preprocessing import binarize
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import plot_importance
from sklearn import metrics

In [11]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# setting format for readability
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [12]:
df = pd.read_pickle('../mod5project/pickle_files/final_data')
print(df.shape)
df.head()

(5492, 106)


,PlayerName,Position,School,CombineYear,Height,Weight,_40yd,Vertical,Bench,BroadJump,_3Cone,Shuttle,Draft Team,Round,Pick,Draft Year,DraftedvNotDrafted,Defense_Games,Solo_Tackles,Assisted_Tackles,Ttl_Tackles,Loss,Sacks,Defensive_Interceptions,Def_Int_Yds,Yds_per_Int,Pick_6,Defended_Passes,Recovered_Fumbles,Rec_Fumbles_Yds,Fumbles_Returned_TD,Forced_Fumbles,Passing_Games,Completions,Pass_Attempts,Completion_Percent,Pass_Yards,Pass_Yds_per_Attempt,Adj_Pass_Yds_per_Attempt,Pass_TDs,Pass_Interceptions,Passer_Rating,CompletionsperGame,Pass_AttemptsperGame,Pass_YardsperGame,Pass_TDsperGame,Pass_InterceptionsperGame,Rush_Rec_Games,Rush_Attempts,Rush_Yds,Rush_Yds_per_Attempt,Rush_TDs,Receptions,Rec_Yds,Rec_Yds_per_Reception,Rec_TDs,Plays_from_Scrimmage,Scrimmage_Yds,Scrimmage_Yds_per_Attempt,Scrimmage_TDs,Rush_AttemptsperGame,Rush_YdsperGame,Rush_TDsperGame,ReceptionsperGame,Rec_YdsperGame,Rec_TDsperGame,Plays_from_ScrimmageperGame,Scrimmage_YdsperGame,Scrimmage_TDsperGame,Kicking_Games,XP_Made,XP_Attempts,XP_Percent,FG_Made,FG_Attempts,FG_Percent,TTL_Kicking_Points,NumPunts,Punt_Yds,Yds_per_Punt,XP_MadeperGame,XP_AttemptsperGame,FG_MadeperGame,FG_AttemptsperGame,TTL_Kicking_PointsperGame,NumPuntsperGame,Punt_YdsperGame,PuntRet_Games,Kickoff_Returns,KO_Return_Yds,Yds_per_KO_Return,KO_Returned_for_TD,Kickoff_ReturnsperGame,KO_Return_YdsperGame,KO_Returned_for_TDperGame,Scoring_Games,TD_Other,Ttl_TDs,TwoPT_Conversion,Safety,TTL_Points,Feet,Inches,Pos,CatchTheQB,NoRespect
0,Shaun Alexander,RB,Alabama,2000,72,218,4.58,17.23,9.70,59.13,3.53,2.13,Seattle Seahawks,1st,19th pick,2000,Drafted,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.09,0.00,0.00,0.00,11.00,302.00,1383.00,4.60,19.00,25.00,323.00,12.90,4.00,327.00,1706.00,5.20,23.00,27.45,125.73,1.73,2.27,29.36,0.36,29.73,155.09,2.09,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11.00,0.00,24.00,0.00,0.00,144.00,72,0,Receivers,False,False
1,LaVar Arrington,OLB,Penn State,2000,75,250,4.53,17.25,11.27,58.80,3.57,2.14,Washington Redskins,1st,2nd pick,2000,Drafted,12.00,0.00,0.00,0.00,0.00,0.00,1.00,27.00,27.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,12.00,0.00,0.00,0.00,0.00,0.00,72,3,LBs,False,False
2,John Baker,P,North Texas,2000,75,227,2.46,15.67,8.09,56.95,3.58,2.21,Not Drafted,Not Drafted,Not Drafted,Not Drafted,Not Drafted,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11.00,1.00,1.00,100.00,27.00,27.00,27.00,0.00,0.00,326.80,0.09,0.09,2.45,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,72,3,SpecialTeams,False,False
3,Anthony Becht,TE,West Virginia,2000,78,270,4.78,33.50,10.11,123.00,6.94,4.08,New York Jets,1st,27th pick,2000,Drafted,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11.00,0.00,5.00,0.00,0.00,30.00,72,6,Receivers,False,False
4,Tom Brady,QB,Michigan,2000,76,211,5.28,24.50,9.61,99.00,7.20,4.38,New England Patriots,6th,199th pick,2000,Drafted,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11

In [13]:
dummies = pd.get_dummies(df['School'], drop_first=True)
df = pd.concat([df, dummies], axis = 1)
dummies = pd.get_dummies(df['Position'], drop_first=True)
df = pd.concat([df, dummies], axis =1)
dummies = pd.get_dummies(df['Pos'], drop_first=True)
df = pd.concat([df, dummies], axis =1)
print(df.shape)
df.head()

(5492, 274)


,PlayerName,Position,School,CombineYear,Height,Weight,_40yd,Vertical,Bench,BroadJump,_3Cone,Shuttle,Draft Team,Round,Pick,Draft Year,DraftedvNotDrafted,Defense_Games,Solo_Tackles,Assisted_Tackles,Ttl_Tackles,Loss,Sacks,Defensive_Interceptions,Def_Int_Yds,Yds_per_Int,Pick_6,Defended_Passes,Recovered_Fumbles,Rec_Fumbles_Yds,Fumbles_Returned_TD,Forced_Fumbles,Passing_Games,Completions,Pass_Attempts,Completion_Percent,Pass_Yards,Pass_Yds_per_Attempt,Adj_Pass_Yds_per_Attempt,Pass_TDs,Pass_Interceptions,Passer_Rating,CompletionsperGame,Pass_AttemptsperGame,Pass_YardsperGame,Pass_TDsperGame,Pass_InterceptionsperGame,Rush_Rec_Games,Rush_Attempts,Rush_Yds,Rush_Yds_per_Attempt,Rush_TDs,Receptions,Rec_Yds,Rec_Yds_per_Reception,Rec_TDs,Plays_from_Scrimmage,Scrimmage_Yds,Scrimmage_Yds_per_Attempt,Scrimmage_TDs,Rush_AttemptsperGame,Rush_YdsperGame,Rush_TDsperGame,ReceptionsperGame,Rec_YdsperGame,Rec_TDsperGame,Plays_from_ScrimmageperGame,Scrimmage_YdsperGame,Scrimmage_TDsperGame,Kicking_Games,XP_Made,XP_Attempts,XP_Percent,FG_Made,FG_Attempts,FG_Percent,TTL_Kicking_Points,NumPunts,Punt_Yds,Yds_per_Punt,XP_MadeperGame,XP_AttemptsperGame,FG_MadeperGame,FG_AttemptsperGame,TTL_Kicking_PointsperGame,NumPuntsperGame,Punt_YdsperGame,PuntRet_Games,Kickoff_Returns,KO_Return_Yds,Yds_per_KO_Return,KO_Returned_for_TD,Kickoff_ReturnsperGame,KO_Return_YdsperGame,KO_Returned_for_TDperGame,Scoring_Games,TD_Other,Ttl_TDs,TwoPT_Conversion,Safety,TTL_Points,Feet,Inches,Pos,CatchTheQB,NoRespect,Akron,Ala-Birmingham,Alabama,Appalachian State,Arizona,Arizona State,Arkansas,Arkansas State,Army,Auburn,BYU,Ball State,Baylor,Boise State,Boston Col.,Boston College,Bowling Green,Buffalo,California,Central Florida,Central Michigan,Charlotte,Cincinnati,Clemson,Colorado,Colorado State,Connecticut,Duke,East Carolina,East. Kentucky,East. Michigan,Florida,Florida Atlantic,Florida International,Florida State,Fresno State,Georgia,Georgia Southern,Georgia State,Georgia Tech,Hawaii,Houston,Idaho,Idaho State,Illinois,Indiana,Iowa,Iowa State,Kansas,Kansas State,Kent State,Kentucky,LSU,La-Monroe,Liberty,Louisiana,Louisiana Tech,Louisiana-Lafayette,Louisville,Marshall,Maryland,Massachusetts,Memphis,Miami,Miami (FL),Miami (OH),Michigan,Michigan State,Middle Tennessee State,Minnesota,Mississippi,Mississippi State,Missouri,Navy,Nebraska,Nevada,New Hampshire,New Mexico,New Mexico State,North Carolina,North Carolina State,North Texas,Northern Arizona,Northern Illinois,Northwestern,Northwestern St. (LA),Notre Dame,Ohio,Ohio State,Oklahoma,Oklahoma State,Old Dominion,Oregon,Oregon State,Penn State,Pittsburgh,Purdue,Rice,Rutgers,SMU,San Diego State,San Jose State,South Alabama,South Carolina,South Florida,Southern Miss,Stanford,Syracuse,TCU,Temple,Tennessee,Texas,Texas A&M,Texas State,Texas Tech,Texas-El Paso,Texas-San Antonio,Toledo,Troy,Tulane,Tulsa,UCF,UCLA,UNLV,USC,Utah,Utah State,Vanderbilt,Virginia,Virginia Tech,Wake Forest,Washington,Washington State,West Virginia,West. Michigan,Western Kentucky,Western Michigan,Wisconsin,Wyoming,CB,DB,DE,DL,DT,EDGE,FB,FS,ILB,K,LB,LS,NT,OG,OL,OLB,OT,P,QB,RB,S,SS,TE,WR,LBs,OffensiveLine,QBs,Receivers,SpecialTeams
0,Shaun Alexander,RB,Alabama,2000,72,218,4.58,17.23,9.70,59.13,3.53,2.13,Seattle Seahawks,1st,19th pick,2000,Drafted,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.09,0.00,0.00,0.00,11.00,302.00,1383.00,4.60,19.00,25.00,323.00,12.90,4.00,327.00,1706.00,5.20,23.00,27.45,125.73,1.73,2.27,29.36,0.36,29.73,155.09,2.09,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11.00,0.00,24.00,0.00,0.00,144.00,72,0,Receivers,False,False,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [61]:
global summary_df
summary_df = pd.DataFrame(columns=['Model', 'Accuracy', 'Precision', 'Recall', 'F1 Score'])
summary_df

,Model,Accuracy,Precision,Recall,F1 Score


## Functions

In [58]:
def train_test(features, target):
    X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=17)
    return X_train, X_test, y_train, y_test

In [59]:
def baseline(features, target):
    X_train, X_test, y_train, y_test = train_test(features, target)
    
    model = DummyClassifier(strategy='uniform', random_state=17)
    model = model.fit(X_train, y_train)
    return model

In [64]:
def scoring(model, features, target):
    model_name = str(model)
    model_name = model_name.lstrip('< function ')
    model_name = model_name.rstrip(' at')
    
    X_train, X_test, y_train, y_test = train_test(features, target)
    model = model(features, target)
    
    test_predictions = model.predict(X_test)
    
    accuracy = metrics.accuracy_score(y_test, test_predictions)
    precision = metrics.precision_score(y_test, test_predictions, average='weighted')
    recall = metrics.recall_score(y_test, test_predictions, average='weighted')
    f1_score = metrics.f1_score(y_test, test_predictions, average='weighted')
    
    global summary_df
    summary_df = summary_df.append({'Model': model_name, 
                                    'Accuracy': accuracy, 
                                    'Precision': precision, 
                                    'Recall': recall, 
                                    'F1 Score': f1_score}, 
                                   ignore_index=True)
    
    return summary_df

In [65]:
scoring(baseline, all_features_w_dummies, target)

,Model,Accuracy,Precision,Recall,F1 Score
0,baseline at 0x1a1d8c9950>,0.49,0.57,0.49,0.51
1,baseline at 0x1a1d8c9950>,0.49,0.57,0.49,0.51


In [49]:
def decision_tree(features, target):
    X_train, X_test, y_train, y_test = train_test(features, target)
    
    model = DecisionTreeClassifier(random_state=17)
    model = model.fit(X_train, y_train)
    return model

In [50]:
scoring(decision_tree, all_features_w_dummies, target)

,Model,Accuracy,Precision,Recall,F1 Score
0,<function baseline at 0x1a1d58b440>,0.49,nan,nan,nan
1,<function baseline at 0x1a1d58b440>,0.49,nan,nan,nan
2,<function baseline at 0x1a1d58b440>,0.49,0.57,0.49,0.51
3,<function decision_tree at 0x1a1d8c9290>,0.67,0.68,0.67,0.67


In [51]:
dec_tree = decision_tree(all_features_w_dummies, target)

## Setting Target & Features

In [18]:
print(len(list(df.columns)))
str(list(df.columns))

274


"['PlayerName', 'Position', 'School', 'CombineYear', 'Height', 'Weight', '_40yd', 'Vertical', 'Bench', 'BroadJump', '_3Cone', 'Shuttle', 'Draft Team', 'Round', 'Pick', 'Draft Year', 'DraftedvNotDrafted', 'Defense_Games', 'Solo_Tackles', 'Assisted_Tackles', 'Ttl_Tackles', 'Loss', 'Sacks', 'Defensive_Interceptions', 'Def_Int_Yds', 'Yds_per_Int', 'Pick_6', 'Defended_Passes', 'Recovered_Fumbles', 'Rec_Fumbles_Yds', 'Fumbles_Returned_TD', 'Forced_Fumbles', 'Passing_Games', 'Completions', 'Pass_Attempts', 'Completion_Percent', 'Pass_Yards', 'Pass_Yds_per_Attempt', 'Adj_Pass_Yds_per_Attempt', 'Pass_TDs', 'Pass_Interceptions', 'Passer_Rating', 'CompletionsperGame', 'Pass_AttemptsperGame', 'Pass_YardsperGame', 'Pass_TDsperGame', 'Pass_InterceptionsperGame', 'Rush_Rec_Games', 'Rush_Attempts', 'Rush_Yds', 'Rush_Yds_per_Attempt', 'Rush_TDs', 'Receptions', 'Rec_Yds', 'Rec_Yds_per_Reception', 'Rec_TDs', 'Plays_from_Scrimmage', 'Scrimmage_Yds', 'Scrimmage_Yds_per_Attempt', 'Scrimmage_TDs', 'Rush_Atte

In [5]:
target = df['DraftedvNotDrafted']

In [22]:
all_features_w_dummies = df.drop(columns=['PlayerName', 'Position', 'School', 'CombineYear', 'Draft Team', 'Round', 
                                           'Pick', 'Draft Year', 'DraftedvNotDrafted', 'Feet', 'Inches', 'Pos'])

short_feat_w_dummies = df.drop(columns=['PlayerName', 'Position', 'School', 'CombineYear', 'Draft Team', 'Round', 
                                        'Pick', 'Draft Year', 'DraftedvNotDrafted', 'Feet', 'Inches', 'Pos', 
                                        'Completions', 'Pass_Attempts', 'Completion_Percent', 'Pass_Yards', 
                                        'Pass_Yds_per_Attempt', 'Adj_Pass_Yds_per_Attempt', 'Pass_TDs', 
                                        'Pass_Interceptions', 'Passer_Rating', 'CompletionsperGame', 
                                        'Pass_AttemptsperGame', 'Pass_YardsperGame', 'Pass_TDsperGame', 
                                        'Pass_InterceptionsperGame', 'Rush_Attempts', 'Rush_Yds',                                         
                                        'Rush_Yds_per_Attempt', 'Rush_TDs', 'Receptions', 'Rec_Yds', 
                                        'Rec_Yds_per_Reception', 'Rec_TDs', 'Plays_from_Scrimmage', 
                                        'Scrimmage_Yds', 'Scrimmage_Yds_per_Attempt', 'Scrimmage_TDs', 
                                        'Rush_AttemptsperGame', 'Rush_YdsperGame', 'Rush_TDsperGame', 
                                        'ReceptionsperGame', 'Rec_YdsperGame', 'Rec_TDsperGame', 
                                        'Plays_from_ScrimmageperGame', 'Scrimmage_YdsperGame', 
                                        'Scrimmage_TDsperGame', 'Yds_per_KO_Return', 'KO_Returned_for_TD', 
                                        'KO_Return_YdsperGame', 'KO_Returned_for_TDperGame', 'Scoring_Games', 
                                        'TD_Other', 'TwoPT_Conversion', 'Safety', 'TTL_Points'])

all_features_no_dummies = df.drop(columns=['PlayerName', 'Position', 'School', 'CombineYear', 'Draft Team', 'Round', 
                                           'Pick', 'Draft Year', 'DraftedvNotDrafted', 'Feet', 'Inches', 'Pos', 
                                           'Akron', 'Ala-Birmingham', 'Alabama', 'Appalachian State', 'Arizona', 
                                           'Arizona State', 'Arkansas', 'Arkansas State', 'Army', 'Auburn', 'BYU', 
                                           'Ball State', 'Baylor', 'Boise State', 'Boston Col.', 'Boston College', 
                                           'Bowling Green', 'Buffalo', 'California', 'Central Florida', 
                                           'Central Michigan', 'Charlotte', 'Cincinnati', 'Clemson', 'Colorado', 
                                           'Colorado State', 'Connecticut', 'Duke', 'East Carolina', 'East. Kentucky',
                                           'East. Michigan', 'Florida', 'Florida Atlantic', 'Florida International', 
                                           'Florida State', 'Fresno State', 'Georgia', 'Georgia Southern', 
                                           'Georgia State', 'Georgia Tech', 'Hawaii', 'Houston', 'Idaho', 
                                           'Idaho State', 'Illinois', 'Indiana', 'Iowa', 'Iowa State', 'Kansas', 
                                           'Kansas State', 'Kent State', 'Kentucky', 'LSU', 'La-Monroe', 'Liberty', 
                                           'Louisiana', 'Louisiana Tech', 'Louisiana-Lafayette', 'Louisville', 
                                           'Marshall', 'Maryland', 'Massachusetts', 'Memphis', 'Miami', 'Miami (FL)',
                                           'Miami (OH)', 'Michigan', 'Michigan State', 'Middle Tennessee State', 
                                           'Minnesota', 'Mississippi', 'Mississippi State', 'Missouri', 'Navy', 
                                           'Nebraska', 'Nevada', 'New Hampshire', 'New Mexico', 'New Mexico State', 
                                           'North Carolina', 'North Carolina State', 'North Texas', 
                                           'Northern Arizona', 'Northern Illinois', 'Northwestern',
                                           'Northwestern St. (LA)', 'Notre Dame', 'Ohio', 'Ohio State', 'Oklahoma', 
                                           'Oklahoma State', 'Old Dominion', 'Oregon', 'Oregon State', 'Penn State', 
                                           'Pittsburgh', 'Purdue', 'Rice', 'Rutgers', 'SMU', 'San Diego State', 
                                           'San Jose State', 'South Alabama', 'South Carolina', 'South Florida', 
                                           'Southern Miss', 'Stanford', 'Syracuse', 'TCU', 'Temple', 'Tennessee', 
                                           'Texas', 'Texas A&M', 'Texas State', 'Texas Tech', 'Texas-El Paso', 
                                           'Texas-San Antonio', 'Toledo', 'Troy', 'Tulane', 'Tulsa', 'UCF', 'UCLA', 
                                           'UNLV', 'USC', 'Utah', 'Utah State', 'Vanderbilt', 'Virginia', 
                                           'Virginia Tech', 'Wake Forest', 'Washington', 'Washington State', 
                                           'West Virginia', 'West. Michigan', 'Western Kentucky', 'Western Michigan',
                                           'Wisconsin', 'Wyoming', 'CB', 'DB', 'DE', 'DL', 'DT', 'EDGE', 'FB', 'FS', 
                                           'ILB', 'K', 'LB', 'LS', 'NT', 'OG', 'OL', 'OLB', 'OT', 'P', 'QB', 'RB', 
                                           'S', 'SS', 'TE', 'WR', 'LBs', 'OffensiveLine', 'QBs', 'Receivers', 
                                           'SpecialTeams'])

short_feat_no_dummies = df.drop(columns=['PlayerName', 'Position', 'School', 'CombineYear', 'Draft Team', 'Round', 
                                        'Pick', 'Draft Year', 'DraftedvNotDrafted', 'Feet', 'Inches', 'Pos', 
                                        'Completions', 'Pass_Attempts', 'Completion_Percent', 'Pass_Yards', 
                                        'Pass_Yds_per_Attempt', 'Adj_Pass_Yds_per_Attempt', 'Pass_TDs', 
                                        'Pass_Interceptions', 'Passer_Rating', 'CompletionsperGame', 
                                        'Pass_AttemptsperGame', 'Pass_YardsperGame', 'Pass_TDsperGame', 
                                        'Pass_InterceptionsperGame', 'Rush_Attempts', 'Rush_Yds',                                         
                                        'Rush_Yds_per_Attempt', 'Rush_TDs', 'Receptions', 'Rec_Yds', 
                                        'Rec_Yds_per_Reception', 'Rec_TDs', 'Plays_from_Scrimmage', 
                                        'Scrimmage_Yds', 'Scrimmage_Yds_per_Attempt', 'Scrimmage_TDs', 
                                        'Rush_AttemptsperGame', 'Rush_YdsperGame', 'Rush_TDsperGame', 
                                        'ReceptionsperGame', 'Rec_YdsperGame', 'Rec_TDsperGame', 
                                        'Plays_from_ScrimmageperGame', 'Scrimmage_YdsperGame', 
                                        'Scrimmage_TDsperGame', 'Yds_per_KO_Return', 'KO_Returned_for_TD', 
                                        'KO_Return_YdsperGame', 'KO_Returned_for_TDperGame', 'Scoring_Games', 
                                        'TD_Other', 'TwoPT_Conversion', 'Safety', 'TTL_Points', 'Akron', 
                                        'Ala-Birmingham', 'Alabama', 'Appalachian State', 'Arizona', 
                                        'Arizona State', 'Arkansas', 'Arkansas State', 'Army', 'Auburn', 'BYU', 
                                        'Ball State', 'Baylor', 'Boise State', 'Boston Col.', 'Boston College', 
                                        'Bowling Green', 'Buffalo', 'California', 'Central Florida', 
                                        'Central Michigan', 'Charlotte', 'Cincinnati', 'Clemson', 'Colorado', 
                                        'Colorado State', 'Connecticut', 'Duke', 'East Carolina', 'East. Kentucky',
                                        'East. Michigan', 'Florida', 'Florida Atlantic', 'Florida International', 
                                        'Florida State', 'Fresno State', 'Georgia', 'Georgia Southern', 
                                        'Georgia State', 'Georgia Tech', 'Hawaii', 'Houston', 'Idaho', 
                                        'Idaho State', 'Illinois', 'Indiana', 'Iowa', 'Iowa State', 'Kansas', 
                                        'Kansas State', 'Kent State', 'Kentucky', 'LSU', 'La-Monroe', 'Liberty', 
                                        'Louisiana', 'Louisiana Tech', 'Louisiana-Lafayette', 'Louisville', 
                                        'Marshall', 'Maryland', 'Massachusetts', 'Memphis', 'Miami', 'Miami (FL)',
                                        'Miami (OH)', 'Michigan', 'Michigan State', 'Middle Tennessee State', 
                                        'Minnesota', 'Mississippi', 'Mississippi State', 'Missouri', 'Navy', 
                                        'Nebraska', 'Nevada', 'New Hampshire', 'New Mexico', 'New Mexico State', 
                                        'North Carolina', 'North Carolina State', 'North Texas', 
                                        'Northern Arizona', 'Northern Illinois', 'Northwestern',
                                        'Northwestern St. (LA)', 'Notre Dame', 'Ohio', 'Ohio State', 'Oklahoma', 
                                        'Oklahoma State', 'Old Dominion', 'Oregon', 'Oregon State', 'Penn State', 
                                        'Pittsburgh', 'Purdue', 'Rice', 'Rutgers', 'SMU', 'San Diego State', 
                                        'San Jose State', 'South Alabama', 'South Carolina', 'South Florida', 
                                        'Southern Miss', 'Stanford', 'Syracuse', 'TCU', 'Temple', 'Tennessee', 
                                        'Texas', 'Texas A&M', 'Texas State', 'Texas Tech', 'Texas-El Paso', 
                                        'Texas-San Antonio', 'Toledo', 'Troy', 'Tulane', 'Tulsa', 'UCF', 'UCLA', 
                                        'UNLV', 'USC', 'Utah', 'Utah State', 'Vanderbilt', 'Virginia', 
                                        'Virginia Tech', 'Wake Forest', 'Washington', 'Washington State', 
                                        'West Virginia', 'West. Michigan', 'Western Kentucky', 'Western Michigan',
                                        'Wisconsin', 'Wyoming', 'CB', 'DB', 'DE', 'DL', 'DT', 'EDGE', 'FB', 'FS', 
                                        'ILB', 'K', 'LB', 'LS', 'NT', 'OG', 'OL', 'OLB', 'OT', 'P', 'QB', 'RB', 
                                        'S', 'SS', 'TE', 'WR', 'LBs', 'OffensiveLine', 'QBs', 'Receivers', 
                                        'SpecialTeams'])


DummyClassifier(constant=None, random_state=17, strategy='uniform')